normalizing the images using global visulaization.

In [1]:
import cv2
import numpy as np
import os

def normalize_image(image, global_min, global_max):
    """Normalize image using global min-max scaling to [0,1]."""
    return (image - global_min) / (global_max - global_min + 1e-8)

# Define input and output folders
folders = {
    "processed_seis": "normalized_seis",
    "processed_fault": "normalized_fault",
    "augmented_seis": "normalized_aug_seis",
    "augmented_fault": "normalized_aug_fault",
    "sliced_validation/seis": "normalized_validation/seis",
    "sliced_validation/fault": "normalized_validation/fault"
}

# Create output folders if they don’t exist
for out_folder in folders.values():
    os.makedirs(out_folder, exist_ok=True)

# Compute global min/max from training images
train_min, train_max = np.inf, -np.inf
train_folders = ["processed_seis", "processed_fault", "augmented_seis", "augmented_fault"]

for folder in train_folders:
    for img_name in os.listdir(folder):
        if img_name.endswith(".png"):
            img = cv2.imread(os.path.join(folder, img_name), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                train_min = min(train_min, img.min())
                train_max = max(train_max, img.max())

print(f"🌍 Global normalization: min={train_min}, max={train_max}")

# Normalize and save images
for in_folder, out_folder in folders.items():
    for img_name in os.listdir(in_folder):
        if img_name.endswith(".png"):
            img_path = os.path.join(in_folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                print(f"❌ Skipping {img_name} (loading error)")
                continue
            
            # Normalize
            norm_img = normalize_image(img, train_min, train_max)
            
            # Convert back to 8-bit (0-255) for saving
            norm_img = (norm_img * 255).astype(np.uint8)
            
            # Save
            cv2.imwrite(os.path.join(out_folder, img_name), norm_img)

print("✅ Global Normalization Completed! Check the normalized folders.")


🌍 Global normalization: min=0, max=255
✅ Global Normalization Completed! Check the normalized folders.
